In [1]:
# turn on autoreload
%load_ext autoreload
%autoreload 2

In [20]:
import os, logging, sys
from dotenv import load_dotenv, find_dotenv
from datetime import date
load_dotenv(find_dotenv(), override=True)

ROOT = os.path.abspath("..")          # if your notebook/ lives under project-root/notebook/
SRC  = os.path.join(ROOT, "src")
sys.path.append(SRC)


# quiet httpx unless you want network noise
#logging.getLogger("httpx").setLevel(logging.WARNING)

#from agents import Runner

from agent_strategy import strategy_agent
from agent_pricer import pricer_agent
from agent_coach import coach_agent
from tools_parser import parse_and_store_mvp, parse_and_store_mvp_impl, normalize_and_store_legs_impl
from bus import BUS                      # your file-backed KV/log bus

In [ ]:
ref_date=date.today()
txt="parse strategy: translate this request: i want to trade 500 AAPL jan26 250 300 collars"
payload = txt.split(":", 1)[1].strip()
out = parse_and_store_mvp_impl(text=payload)  # direct Python call
out2=normalize_and_store_legs_impl(key=out["key"], ref_year=ref_date.year, ref_month=ref_date.month, ref_day=ref_date.day)
a=bus_get("legs",out2["key"])
legs=a["payload"]["legs"]

{
  "status": "done",
  "producer": "parser_py",
  "payload": {
    "ok": true,
    "quantity": 500,
    "symbol": "AAPL",
    "maturities": [
      "Jan26"
    ],
    "strikes": [
      250.0,
      300.0
    ],
    "ratio": "1x1",
    "structure": "custom",
    "legs": {
      "legs": [
        {
          "cp": "P",
          "strike": 250.0,
          "side": "AUTO",
          "ratio": 1,
          "qty": 500,
          "expiry": {
            "year": 2026,
            "month": 1,
            "iso": "2026-01-16",
            "label": "Jan26"
          }
        },
        {
          "cp": "C",
          "strike": 300.0,
          "side": "AUTO",
          "ratio": 1,
          "qty": 500,
          "expiry": {
            "year": 2026,
            "month": 1,
            "iso": "2026-01-16",
            "label": "Jan26"
          }
        }
      ],
      "issues": [],
      "notes": ""
    }
  }
}


{'legs': [{'cp': 'P',
   'strike': 250.0,
   'side': 'AUTO',
   'ratio': 1,
   'qty': 500,
   'expiry': {'year': 2026,
    'month': 1,
    'iso': '2026-01-16',
    'label': 'Jan26'}},
  {'cp': 'C',
   'strike': 300.0,
   'side': 'AUTO',
   'ratio': 1,
   'qty': 500,
   'expiry': {'year': 2026,
    'month': 1,
    'iso': '2026-01-16',
    'label': 'Jan26'}}],
 'issues': [],
 'notes': ''}

In [22]:
out2

{'ok': True, 'key': 'AAPL-Jan26-custom', 'issues': []}

In [24]:
from typing import Optional
import json
import time
def bus_list(topic: str):
    keys = BUS.list_keys(topic)
    print(f"[{topic}] {len(keys)} key(s)")
    for k in keys:
        print("  -", k)

def bus_get(topic: str, key: str, path: Optional[str]=None):
    """Pretty-print an entry; optional 'path' lets you dive into payload (e.g., 'result', 'payload.result')."""
    val = BUS.get(topic=topic, key=key)
    if val is None:
        print(f"⛔ not found: {topic}:{key}")
        return None
    obj = val
    if path:
        for part in path.split("."):
            if isinstance(obj, dict) and part in obj:
                obj = obj[part]
            else:
                print(f"⚠️ path '{path}' not found; showing full object"); obj = val; break
    print(json.dumps(obj, indent=2, default=str)[:2000])
    return obj

def bus_put(topic: str, key: str, payload: dict, producer="notebook"):
    BUS.put(topic=topic, key=key, payload=payload, producer=producer)
    print(f"✅ wrote {topic}:{key}")

def bus_del(topic: str, key: str):
    # soft-delete: write a tombstone-like payload
    BUS.put(topic=topic, key=key, payload={"status":"deleted","ts":time.time()}, producer="notebook")
    print(f"🗑️  marked deleted {topic}:{key}")

def bus_clear_topic(topic: str):
    for k in BUS.list_keys(topic):
        bus_del(topic, k)

In [15]:
from datetime import date
from os import forkpty
from numpy.random import f
import pandas as pd


def numify(df, col, dtype="float64"):
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype(dtype)
    else:
        df[col] = pd.Series(pd.NA, index=df.index, dtype=dtype)


def collect_market_and_resolve(
    symbol: str,
    trade_date: str,                    # 'YYYY-MM-DD'
    requested_legs: list[dict],         # [{"cp":"P","strike":250,"expiry":{"year":2026,"month":1,"iso":None}}, ...]
    getMarketData,                      # ivolatility method: setMethod('/equities/eod/stock-opts-by-param')
    dte_from: int = 0,
    dte_to: int = 760,
    mny_from: int = -90,
    mny_to: int = 90,
) -> dict:
    """
    Returns:
      {
        "ok": bool,
        "dataset": [ {optionId, symbol, cp, expiration, strike, bid, ask, iv, ...} ],
        "issues": [ ... ],
        "chain_ref": df  # cleaned DataFrame for charts
      }
    """
    issues = []
    sym = symbol.upper()

    # 1) Bulk fetch calls & puts
    calls = getMarketData(symbol=sym, tradeDate=trade_date, dteFrom=dte_from, dteTo=dte_to,
                          moneynessFrom=mny_from, moneynessTo=mny_to, cp='C')
    puts  = getMarketData(symbol=sym, tradeDate=trade_date, dteFrom=dte_from, dteTo=dte_to,
                          moneynessFrom=mny_from, moneynessTo=mny_to, cp='P')
    df = pd.concat([calls, puts], ignore_index=True) if not (calls.empty and puts.empty) else pd.DataFrame()
    if df.empty:
        return {"ok": False, "dataset": [], "issues": [{"code":"NO_CHAIN","msg":"no data returned"}], "chain_ref": df}

    # 2) Clean/normalize columns we’ll use
    colmap = {
        "expiration_date":"expiration", "call_put":"cp", "price_strike":"strike",
        "Bid":"bid", "Ask":"ask", "iv":"iv", "option_id":"optionId", "option_symbol":"osym",
        "openinterest":"openInterest", "volume":"volume"
    }
    df = df.rename(columns={k:v for k,v in colmap.items() if k in df.columns}).copy()
    df["cp"]        = df["cp"].str.upper().str[0]
    df["is_settlement"]= df.get("is_settlement", 0).fillna(0)

    for field in ["bid", "ask", "iv", "strike"]:
        numify(df, field)


    for field in ["openInterest", "volume"]:
        numify(df, field, dtype="Int64")
        #df[field] = df[field].fillna(0)

    # 3) Index for fast selection: per (exp, cp) keep a sorted strike list
    by_key = {}
    for (exp, cp), g in df.groupby(["expiration","cp"]):
        g = g.sort_values("strike")
        by_key[(exp, cp)] = g

    # 4) Selection helpers
    def pick_expiry(leg):
        exp = leg.get("expiry") or {}
        iso = exp.get("iso")
        if iso:
            return iso, False  # (chosen, snapped?)
        # fallback month/year → pick nearest listed expiry in that month, else closest overall
        y, m = exp.get("year"), exp.get("month")
        if y and m:
            month_listed = sorted({e for (e,cp) in by_key.keys() if int(e[:4])==y and int(e[5:7])==m})
            if month_listed:
                # prefer the earliest in-month expiry (often weeklies + monthly)
                return month_listed[0], False
        # closest overall to intended 3rd Friday if available
        listed = sorted({e for (e,cp) in by_key.keys()})
        if not listed:
            return None, False
        # crude “closest” by string distance proxy (lex order ≈ chronological for ISO)
        # better: convert to dates and abs delta; left as simple for MVP
        return listed[0], True

    def pick_strike(exp: str, cp: str, target_k: float):
        g = by_key.get((exp, cp))
        if g is None or g.empty:
            return None, None, None, False
        strikes = g["strike"].values
        # nearest by absolute distance
        import numpy as np
        idx = int(np.argmin(np.abs(strikes - target_k)))
        chosen = strikes[idx]
        below = strikes[max(idx-1, 0)]
        above = strikes[min(idx+1, len(strikes)-1)]
        exact = float(chosen) == float(target_k)
        return float(chosen), float(below), float(above), exact

    def best_row(exp: str, cp: str, strike: float):
        g = by_key.get((exp, cp))
        if g is None: return None
        sel = g[g["strike"] == strike]
        if sel.empty: return None
        # prefer non-settlement, then highest OI, then volume
        sel = sel.sort_values(by=["is_settlement","openInterest","volume"], ascending=[True, False, False])
        return sel.iloc[0].to_dict()

    # 5) Resolve each requested leg
    rows = []
    for i, leg in enumerate(requested_legs):
        cp  = str(leg["cp"]).upper()[0]
        k   = float(leg["strike"] if "strike" in leg else leg.get("k"))
        exp, month_snap = pick_expiry(leg)
        if not exp:
            issues.append({"code":"NO_EXP","msg":f"leg {i}: no expiry available"}); continue

        chosen, below, above, exact = pick_strike(exp, cp, k)
        if chosen is None:
            issues.append({"code":"NO_STRIKE","msg":f"leg {i}: no strikes for {cp} @ {exp}"}); continue
        if not exact:
            issues.append({"code":"STRIKE_ADJUSTED","msg":f"leg {i}: {k} -> {chosen} for {cp} @ {exp}", "details":{"below":below,"above":above}})

        r = best_row(exp, cp, chosen)
        if r is None:
            issues.append({"code":"ROW_MISSING","msg":f"leg {i}: missing row {cp} {chosen} @ {exp}"}); continue

        rows.append({
            "optionId": r.get("optionId") or r.get("osym") or f"{sym}:{exp}:{cp}:{int(round(chosen))}",
            "symbol": sym, "cp": cp, "expiration": exp, "strike": float(chosen),
            "bid": r.get("bid"), "ask": r.get("ask"), "iv": r.get("iv"),
            "openInterest": r.get("openInterest"), "volume": r.get("volume"),
            "delta":r.get("delta"),
            "gamma":r.get("gamma"),
            "vega":r.get("vega"),
            "underlying_price":r.get("underlying_price")
        })

    return {"ok": len(rows)==len(requested_legs), "dataset": rows, "issues": issues, "chain_ref": df}

In [11]:
import dotenv
import os
import ivolatility as ivol

dotenv.load_dotenv()
api_key = os.getenv("IVOL_API_KEY")
if not api_key:
    raise RuntimeError("❌ IVOL_API_KEY not found in .env file or environment.")

ivol.setLoginParams(apiKey=api_key)
api_key

getMarketData = ivol.setMethod('/equities/eod/stock-opts-by-param')

In [ ]:
marketData = getMarketData(symbol='AAPL',tradeDate='2025-09-25',dteFrom=0,dteTo=760,moneynessFrom=-100,moneynessTo=100,cp='C')
marketData_puts=getMarketData(symbol='AAPL',tradeDate='2025-09-25',dteFrom=0,dteTo=760,moneynessFrom=-100,moneynessTo=100,cp='P')
marketData=pd.concat([marketData,marketData_puts])


KeyError: 'openInterest'

In [87]:
marketData["openinterest"]

0       17
1       20
2       12
3       10
4        5
        ..
1090     0
1091     0
1092     0
1093     0
1094     0
Name: openinterest, Length: 2190, dtype: int64

In [16]:

requested_legs = [
  {"cp":"P","strike":250.0,"expiry":{"year":2026,"month":1}},  # Jan26 P250
  {"cp":"C","strike":300.0,"expiry":{"year":2026,"month":1}},  # Jan26 C300
]

out = collect_market_and_resolve("AAPL", "2025-10-07", requested_legs, getMarketData)
out["ok"], out["issues"], out["dataset"][:2]
chain_df = out["chain_ref"]   # for smiles/term-structure charts

In [17]:
chain_df[chain_df["expiration"]=="2026-01-16"].columns

Index(['c_date', 'osym', 'dte', 'stocks_id', 'expiration', 'cp', 'strike',
       'price_open', 'price_high', 'price_low', 'price', 'volume',
       'openInterest', 'iv', 'delta', 'preiv', 'gamma', 'theta', 'vega', 'rho',
       'ask', 'bid', 'underlying_price', 'calc_OTM', 'optionId',
       'is_settlement'],
      dtype='object')

In [18]:
out

{'ok': True,
 'dataset': [{'optionId': 128416790,
   'symbol': 'AAPL',
   'cp': 'P',
   'expiration': '2026-01-16',
   'strike': 250.0,
   'bid': 9.0,
   'ask': 9.15,
   'iv': 0.244109,
   'openInterest': 16611,
   'volume': 318,
   'delta': -0.375373,
   'gamma': 0.011914,
   'vega': 0.513147,
   'underlying_price': 256.48},
  {'optionId': 128416807,
   'symbol': 'AAPL',
   'cp': 'C',
   'expiration': '2026-01-16',
   'strike': 300.0,
   'bid': 1.75,
   'ask': 1.79,
   'iv': 0.228176,
   'openInterest': 47201,
   'volume': 1465,
   'delta': 0.122472,
   'gamma': 0.006597,
   'vega': 0.273568,
   'underlying_price': 256.48}],
 'issues': [],
 'chain_ref':           c_date                   osym  dte  stocks_id  expiration cp  \
 0     2025-10-07  AAPL  251010C00110000    3        799  2025-10-10  C   
 1     2025-10-07  AAPL  251010C00120000    3        799  2025-10-10  C   
 2     2025-10-07  AAPL  251010C00125000    3        799  2025-10-10  C   
 3     2025-10-07  AAPL  251010C001300

In [12]:
bus_list("legs")
#bus_get("spec", 'AAPL-Jan26-custom') 


[legs] 1 key(s)
  - AAPL-Jan26-custom


In [9]:
bus_get("spec", 'AAPL-Jan26-custom')

{
  "status": "done",
  "producer": "parser_py",
  "payload": {
    "ok": true,
    "quantity": 500,
    "symbol": "AAPL",
    "maturities": [
      "Jan26"
    ],
    "strikes": [
      250.0,
      300.0
    ],
    "ratio": "1x1",
    "structure": "custom",
    "legs": {
      "legs": [
        {
          "cp": "P",
          "strike": 250.0,
          "side": "AUTO",
          "ratio": 1,
          "qty": 500,
          "expiry": {
            "year": 2026,
            "month": 1,
            "iso": "2026-01-16",
            "label": "Jan26"
          }
        },
        {
          "cp": "C",
          "strike": 300.0,
          "side": "AUTO",
          "ratio": 1,
          "qty": 500,
          "expiry": {
            "year": 2026,
            "month": 1,
            "iso": "2026-01-16",
            "label": "Jan26"
          }
        }
      ],
      "issues": [],
      "notes": ""
    }
  }
}


{'status': 'done',
 'producer': 'parser_py',
 'payload': {'ok': True,
  'quantity': 500,
  'symbol': 'AAPL',
  'maturities': ['Jan26'],
  'strikes': [250.0, 300.0],
  'ratio': '1x1',
  'structure': 'custom',
  'legs': {'legs': [{'cp': 'P',
     'strike': 250.0,
     'side': 'AUTO',
     'ratio': 1,
     'qty': 500,
     'expiry': {'year': 2026,
      'month': 1,
      'iso': '2026-01-16',
      'label': 'Jan26'}},
    {'cp': 'C',
     'strike': 300.0,
     'side': 'AUTO',
     'ratio': 1,
     'qty': 500,
     'expiry': {'year': 2026,
      'month': 1,
      'iso': '2026-01-16',
      'label': 'Jan26'}}],
   'issues': [],
   'notes': ''}}}